In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
import hardware_control.wx_programs as wx
import hardware_control.bnc as bnc
from classes.generator import *
from hardware_control.hardware_config import *
from experiment_configuration.values import *
from classes.qubit_class import *
import daq.daq_programs_homo as daq
import seaborn as sns
import standard_sequences.pinopi as pnp
import analysis.plotting as plotting

ModuleNotFoundError: No module named 'usbtmc'

In [ ]:
q1 = Qubit(q1_dict, readout_dict)
q2 = Qubit(q2_dict, readout_dict)
general_vals_dict["wx_offs"] = [0, 0, -0.08, 0]
readout = Readout(readout_dict)
print(f"{q1}\n{q2}")

In [ ]:
def pi_nopipi_twpa_loss(twpa_freq, twpa_power, reps=10000):
    """
    Loss function to optimize TWPA frequency and power by maximizing SNR.
    Computes SNR from raw IQ data using sample mean and std, no fitting.

    Parameters:
        twpa_freq (float): Frequency of the TWPA (Hz)
        twpa_power (float): Power into the TWPA (dBm)
        reps (int): Number of repetitions per pattern (default: 10,000)

    Returns:
        float: Negative SNR (loss to minimize)
    """
    # Set TWPA hardware parameters
    hardware.set_twpa_frequency(twpa_freq)
    hardware.set_twpa_power(twpa_power)

    # Prepare measurement patterns
    pnp.pi_nopi_ge(0, 0, q2, q1, general_vals_dict)
    pnp.pi_nopi_ge(1, 1, q2, q1, general_vals_dict)
    wx.wx_set_and_amplitude_and_offset(
        amp=general_vals_dict["wx_amps"], offset=general_vals_dict["wx_offs"]
    )

    # Run DAQ
    values = daq.run_daq_het_2q(q1, q2, num_patterns=3, num_records_per_pattern=reps)

    # Extract raw IQ data (assuming 2D array: [2, reps, 2])
    readout_vs_pats = values.rec_readout_vs_pats_2

    # Compute SNR directly
    I_g = readout_vs_pats[0, :, 0]
    I_e = readout_vs_pats[0, :, 1]
    Q_g = readout_vs_pats[1, :, 0]
    Q_e = readout_vs_pats[1, :, 1]

    mu_I_g, mu_I_e = np.mean(I_g), np.mean(I_e)
    mu_Q_g, mu_Q_e = np.mean(Q_g), np.mean(Q_e)
    std_I_g, std_I_e = np.std(I_g), np.std(I_e)
    std_Q_g, std_Q_e = np.std(Q_g), np.std(Q_e)

    signal = np.sqrt((mu_I_g - mu_I_e)**2 + (mu_Q_g - mu_Q_e)**2)
    noise = (np.abs(std_I_g) + np.abs(std_I_e) + np.abs(std_Q_g) + np.abs(std_Q_e)) / 4
    SNR = signal / noise

    # Return negative SNR as a loss to minimize
    return -SNR



NameError: name 'pnp' is not defined